Programa de biblioteca

In [ ]:
from datetime import datetime, timedelta

class Material:
    def __init__(self, titulo, estado="disponible"):
        self.titulo = titulo
        self.estado = estado
        self.sucursal_actual = None

    def cambiar_estado(self, nuevo_estado):
        self.estado = nuevo_estado

    def asignar_sucursal(self, sucursal):
        self.sucursal_actual = sucursal

class Libro(Material):
    def __init__(self, titulo, autor, genero):
        super().__init__(titulo)
        self.autor = autor
        self.genero = genero

class Revista(Material):
    def __init__(self, titulo, edicion, periodicidad):
        super().__init__(titulo)
        self.edicion = edicion
        self.periodicidad = periodicidad

class MaterialDigital(Material):
    def __init__(self, titulo, tipo_archivo, enlace_descarga):
        super().__init__(titulo)
        self.tipo_archivo = tipo_archivo
        self.enlace_descarga = enlace_descarga

class Persona:
    def __init__(self, nombre, id):
        self.nombre = nombre
        self.id = id

class Usuario(Persona):
    def __init__(self, nombre, id):
        super().__init__(nombre, id)
        self.materiales_prestados = []
        self.multas = []

    def consultar_catalogo(self, catalogo, **criterios):
        return catalogo.buscar(**criterios)

    def solicitar_prestamo(self, material, prestamo):
        if material.estado == "disponible" and not self.tiene_multas_pendientes():
            return prestamo.crear_prestamo(self, material)
        return False

    def devolver_material(self, material, prestamo):
        return prestamo.finalizar_prestamo(self, material)

    def tiene_multas_pendientes(self):
        return any(not multa.pagada for multa in self.multas)

class Bibliotecario(Persona):
    def __init__(self, nombre, id, sucursal):
        super().__init__(nombre, id)
        self.sucursal = sucursal

    def agregar_material(self, material):
        self.sucursal.agregar_material(material)

    def gestionar_prestamo(self, usuario, material):
        if material.sucursal_actual == self.sucursal:
            prestamo = Prestamo()
            return prestamo.crear_prestamo(usuario, material)
        return False

    def transferir_material(self, material, sucursal_destino):
        if material.sucursal_actual == self.sucursal:
            self.sucursal.remover_material(material)
            sucursal_destino.agregar_material(material)
            material.asignar_sucursal(sucursal_destino)
            return True
        return False

class Sucursal:
    def __init__(self, nombre, ubicacion):
        self.nombre = nombre
        self.ubicacion = ubicacion
        self.catalogo = Catalogo()
        self.materiales = []

    def agregar_material(self, material):
        self.materiales.append(material)
        material.asignar_sucursal(self)
        self.catalogo.agregar_material(material)

    def remover_material(self, material):
        if material in self.materiales:
            self.materiales.remove(material)
            self.catalogo.remover_material(material)
            return True
        return False

class Prestamo:
    def __init__(self):
        self.fecha_prestamo = None
        self.fecha_devolucion = None
        self.usuario = None
        self.material = None

    def crear_prestamo(self, usuario, material):
        if material.estado == "disponible":
            self.fecha_prestamo = datetime.now()
            self.fecha_devolucion = self.fecha_prestamo + timedelta(days=14)
            self.usuario = usuario
            self.material = material
            material.cambiar_estado("prestado")
            usuario.materiales_prestados.append(material)
            return True
        return False

    def finalizar_prestamo(self, usuario, material):
        if material in usuario.materiales_prestados:
            dias_retraso = self.calcular_dias_retraso()
            if dias_retraso > 0:
                Penalizacion.crear_multa(usuario, dias_retraso)
            material.cambiar_estado("disponible")
            usuario.materiales_prestados.remove(material)
            return True
        return False

    def calcular_dias_retraso(self):
        if datetime.now() > self.fecha_devolucion:
            return (datetime.now() - self.fecha_devolucion).days
        return 0

class Penalizacion:
    MULTA_POR_DIA = 1.0  #$1 por día de retraso

    @staticmethod
    def crear_multa(usuario, dias_retraso):
        multa = {
            'monto': dias_retraso * Penalizacion.MULTA_POR_DIA,
            'pagada': False
        }
        usuario.multas.append(multa)
        return multa

class Catalogo:
    def __init__(self):
        self.materiales = []

    def agregar_material(self, material):
        self.materiales.append(material)

    def remover_material(self, material):
        if material in self.materiales:
            self.materiales.remove(material)

    def buscar(self, **criterios):
        resultados = self.materiales
        
        if 'autor' in criterios:
            resultados = [m for m in resultados if isinstance(m, Libro) and m.autor == criterios['autor']]
        
        if 'genero' in criterios:
            resultados = [m for m in resultados if isinstance(m, Libro) and m.genero == criterios['genero']]
        
        if 'tipo' in criterios:
            resultados = [m for m in resultados if isinstance(m, criterios['tipo'])]

        return resultados

class SistemaBibliotecario:
    def __init__(self):
        self.sucursales = []
    
    def agregar_sucursal(self, sucursal):
        self.sucursales.append(sucursal)
    
    def buscar_en_todas_sucursales(self, **criterios):
        resultados = []
        for sucursal in self.sucursales:
            resultados.extend(sucursal.catalogo.buscar(**criterios))
        return resultados

In [ ]:
#Crear el sistema bibliotecario
sistema = SistemaBibliotecario()

#Crear sucursales
sucursal_centro = Sucursal("Biblioteca Central", "Centro")
sucursal_norte = Sucursal("Biblioteca Norte", "Norte")

#Agregar sucursales al sistema
sistema.agregar_sucursal(sucursal_centro)
sistema.agregar_sucursal(sucursal_norte)

#Crear algunos libros
libro1 = Libro("Cien años de soledad", "Gabriel García Márquez", "Ficción")
libro2 = Libro("El código Da Vinci", "Dan Brown", "Misterio")
libro3 = Libro("Don Quijote", "Miguel de Cervantes", "Clásico")

#Crear bibliotecarios
bibliotecario_centro = Bibliotecario("Ana", "B001", sucursal_centro)
bibliotecario_norte = Bibliotecario("Carlos", "B002", sucursal_norte)

#Agregar libros a las sucursales
bibliotecario_centro.agregar_material(libro1)
bibliotecario_centro.agregar_material(libro2)
bibliotecario_norte.agregar_material(libro3)

#Crear usuario
usuario = Usuario("Juan", "U001")

#búsqueda en todas las sucursales
print("=== Prueba de búsqueda en todas las sucursales ===")
resultados = sistema.buscar_en_todas_sucursales(tipo=Libro)
print(f"Libros encontrados en todas las sucursales:")
for libro in resultados:
    print(f"- {libro.titulo} en {libro.sucursal_actual.nombre}")

#préstamo y penalización
print("\n=== Prueba de préstamo y penalización ===")
prestamo = Prestamo()
#Realizar préstamo
if usuario.solicitar_prestamo(libro1, prestamo):
    print(f"Préstamo exitoso del libro: {libro1.titulo}")
    
    #Simular retraso en la devolución
    prestamo.fecha_devolucion = datetime.now() - timedelta(days=5)  # 5 días de retraso
    
    #Devolver el libro
    if usuario.devolver_material(libro1, prestamo):
        print("Libro devuelto con retraso")
        print(f"Multas del usuario: {len(usuario.multas)}")
        print(f"Monto de la multa: ${usuario.multas[0]['monto']}")

#Prueba de transferencia entre sucursales
print("\n=== Prueba de transferencia entre sucursales ===")
print(f"Ubicación inicial de {libro2.titulo}: {libro2.sucursal_actual.nombre}")

if bibliotecario_centro.transferir_material(libro2, sucursal_norte):
    print(f"Libro transferido exitosamente a: {libro2.sucursal_actual.nombre}")

=== Prueba de búsqueda en todas las sucursales ===
Libros encontrados en todas las sucursales:
- Cien años de soledad en Biblioteca Central
- El código Da Vinci en Biblioteca Central
- Don Quijote en Biblioteca Norte

=== Prueba de préstamo y penalización ===
Préstamo exitoso del libro: Cien años de soledad
Libro devuelto con retraso
Multas del usuario: 1
Monto de la multa: $5.0

=== Prueba de transferencia entre sucursales ===
Ubicación inicial de El código Da Vinci: Biblioteca Central
Libro transferido exitosamente a: Biblioteca Norte
